In [6]:
#using Utk face dataset
#downlaod and sampled upto 60 images for testing the algorithm
#https://www.kaggle.com/datasets/jangedoo/utkface-new

In [2]:
#creating dataframe to store orginal age and gender of the images

In [3]:
import os
import pandas as pd

# Path to the directory containing the image files
directory = 'C://Users/gauth/OneDrive/Desktop/DIP PROJECT/yolo_images'

# List to store extracted information
data = []

# Iterate over files in the directory
for filename in os.listdir(directory):
    # Split filename by ".chip" to get the main part
    main_part = filename.split(".chip")[0]
    
    # Split main part by "_" to extract age and gender
    age, gender, *rest = main_part.split("_")
    
    # Convert gender to string representation
    gender = "Male" if gender == "0" else "Female"
    
    # Join the rest of the filename back together (if any)
    rest = "_".join(rest)
    
    # Append extracted information to the list
    data.append({'Filename': filename, 'Age': int(age), 'Gender': gender})

# Create DataFrame from the extracted information
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


                                 Filename  Age  Gender
0    1_0_0_20161219140623097.jpg.chip.jpg    1    Male
1    1_0_0_20161219140627985.jpg.chip.jpg    1    Male
2    1_0_0_20161219140642920.jpg.chip.jpg    1    Male
3    1_0_0_20161219192208688.jpg.chip.jpg    1    Male
4    1_0_0_20161219205141196.jpg.chip.jpg    1    Male
..                                    ...  ...     ...
62  73_1_0_20170110183953439.jpg.chip.jpg   73  Female
63  73_1_1_20170120230417585.jpg.chip.jpg   73  Female
64  74_0_0_20170120225714392.jpg.chip.jpg   74    Male
65  74_0_0_20170120230055728.jpg.chip.jpg   74    Male
66  74_0_1_20170113182114430.jpg.chip.jpg   74    Male

[67 rows x 3 columns]


In [4]:
#face detection using yolo+predictions using caffe model on utk face images.

In [7]:
import numpy as np
import cv2
import os
import os
from ultralytics import YOLO

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']





folder_path = 'C://Users/gauth/OneDrive/Desktop/DIP PROJECT/yolo_images'

# List all image files in the folder

image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) ]
print(len(image_files))
# Iterate over each image
result_list=[]
model=YOLO("yolov8n-face.pt")
for image_file in image_files:
    # Read image
    img = cv2.imread(image_file)
    results=model(img)
    result_list.append(results)
    boxes=results[0].boxes
    
    
     
    #looping over the faces detected from yolo
    for box in boxes:
        top_left_x=int(box.xyxy.tolist()[0][0])
        top_left_y=int(box.xyxy.tolist()[0][1])
        bottom_right_x=int(box.xyxy.tolist()[0][2])
        bottom_right_y=int(box.xyxy.tolist()[0][3])
        
        cv2.rectangle(img,( top_left_x,top_left_y),(bottom_right_x,  bottom_right_y),(255,0,0),2)
        detected_face=img[top_left_y:bottom_right_y,  top_left_x:bottom_right_x]
        
        detected_face_blob = cv2.dnn.blobFromImage(detected_face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
       
        
        genderNet.setInput(detected_face_blob)
        ageNet.setInput(detected_face_blob)
        
        genderPreds = genderNet.forward()
       
        
        gender = genderList[genderPreds[0].argmax()]
        
        
        agePreds=ageNet.forward()
        
        age = ageList[agePreds[0].argmax()]
        
        
        label = "{}{}".format(gender,age)
       
        cv2.putText(img, label, (top_left_x+30, top_left_y+30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
        
        #loading predicted results to the existing df
        df.loc[df['Filename'] == os.path.basename(image_file), 'Predicted_Gender'] = gender
        df.loc[df['Filename'] == os.path.basename(image_file), 'Predicted_Age'] = age
    cv2.imshow("ok",img)
   
    cv2.waitKey(0)
    

       

cv2.destroyAllWindows()    
        
        
        




67

0: 640x640 2 faces, 408.2ms
Speed: 70.8ms preprocess, 408.2ms inference, 20.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 faces, 230.9ms
Speed: 21.2ms preprocess, 230.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 172.5ms
Speed: 3.4ms preprocess, 172.5ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 177.6ms
Speed: 10.5ms preprocess, 177.6ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 faces, 154.4ms
Speed: 4.8ms preprocess, 154.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 faces, 141.1ms
Speed: 9.1ms preprocess, 141.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 132.1ms
Speed: 6.9ms preprocess, 132.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 faces, 122.2ms
Speed: 7.7ms preprocess, 122.2ms inference, 0.0ms postprocess per ima

Speed: 5.5ms preprocess, 169.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 faces, 148.3ms
Speed: 15.6ms preprocess, 148.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 140.2ms
Speed: 8.0ms preprocess, 140.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


In [8]:
df

,Filename,Age,Gender,Predicted_Gender,Predicted_Age
0,1_0_0_20161219140623097.jpg.chip.jpg,1,Male,Male,(4-6)
1,1_0_0_20161219140627985.jpg.chip.jpg,1,Male,Male,(4-6)
2,1_0_0_20161219140642920.jpg.chip.jpg,1,Male,Male,(4-6)
3,1_0_0_20161219192208688.jpg.chip.jpg,1,Male,Male,(0-2)
4,1_0_0_20161219205141196.jpg.chip.jpg,1,Male,Male,(0-2)
...,...,...,...,...,...
62,73_1_0_20170110183953439.jpg.chip.jpg,73,Female,Male,(25-32)
63,73_1_1_20170120230417585.jpg.chip.jpg,73,Female,NaN,NaN
64,74_0_0_20170120225714392.jpg.chip.jpg,74,Male,Male,(25-32)
65,74_0_0_20170120230055728.jpg.chip.jpg,74,Male,Female,(48-53)


In [9]:
df[30:60]

,Filename,Age,Gender,Predicted_Gender,Predicted_Age
30,29_0_0_20170117180854265.jpg.chip.jpg,29,Male,Male,(4-6)
31,29_0_0_20170117180857403.jpg.chip.jpg,29,Male,Male,(25-32)
32,29_0_0_20170117202637960.jpg.chip.jpg,29,Male,Male,(38-43)
33,29_0_0_20170117202647791.jpg.chip.jpg,29,Male,Male,(25-32)
34,29_0_0_20170117202649479.jpg.chip.jpg,29,Male,Male,(38-43)
35,29_0_0_20170119195119659.jpg.chip.jpg,29,Male,Male,(25-32)
36,31_0_0_20170117133142224.jpg.chip.jpg,31,Male,Male,(0-2)
37,31_0_0_20170117133148137.jpg.chip.jpg,31,Male,Male,(4-6)
38,31_0_0_20170117135213674.jpg.chip.jpg,31,Male,Male,(4-6)
39,31_0_0_20170117135307744.jpg.chip.jpg,31,Male,Male,(38-43)


In [10]:
# Calculate accuracy for predicted age
correct_age_predictions = 0
for index, row in df.iterrows():
    actual_age = row['Age']
    predicted_age_range = row['Predicted_Age']
    if isinstance(predicted_age_range, str):  # Check if the predicted age range is a string
        predicted_age_start, predicted_age_end = map(int, predicted_age_range.strip('()').split('-'))  # Extract start and end of predicted age range
        if actual_age >= predicted_age_start and actual_age <= predicted_age_end:
            correct_age_predictions += 1

total_images = df.shape[0]
age_accuracy = correct_age_predictions / total_images * 100



# Calculate accuracy for predicted gender
correct_gender_predictions = (df['Gender'] == df['Predicted_Gender']).sum()
gender_accuracy = correct_gender_predictions / total_images * 100

# Print results
print("Accuracy for predicted age: {:.2f}%".format(age_accuracy))
print("Accuracy for predicted gender: {:.2f}%".format(gender_accuracy))

Accuracy for predicted age: 26.87%
Accuracy for predicted gender: 59.70%
